# Review data in the parquet cache

Using interactive Panel widgets, query and view the data in the parquet database

## Import python packages

In [1]:
import postevent_dashboard_utils as du
from pathlib import Path
import pandas as pd
import numpy as np
import geopandas as gpd
import panel as pn
import colorcet as cc
import hvplot
import hvplot.pandas
pn.extension()

### Define the set parquet files for each specific study/scenario

For this example, we use dictionaries to define multiple sets of parquet files needed to evaluate operational NWM short- and medium-range streamflow and precipitation forecasts.  As described in previous notebooks, these files define the source of verifying data (primary_filepath), data to evaluate (secondary_filepath), as well as the necessary geometry, crosswalk, and attributes.

In [2]:
# overall study directory
CACHE_DIR = Path(Path.home(), "shared", "teehr-workshop", "post-event-example")

# medium range streamflow forecast evaluation files 
MRF_streamflow = dict(
    scenario_name="medium_range",
    variable="streamflow",
    primary_filepath=Path(CACHE_DIR, "timeseries", "usgs", "*.parquet"),
    secondary_filepath=Path(CACHE_DIR, "timeseries", "medium_range_mem1", "*.parquet"),
    crosswalk_filepath=Path(CACHE_DIR, "geo", "usgs_nwm22_crosswalk.parquet"),
    geometry_filepath=Path(CACHE_DIR, "geo", "usgs_geometry.parquet")
)
# medium range precip forecast evaluation files
MRF_forcing = dict(
    scenario_name="medium_range",
    variable="precipitation",    
    primary_filepath=Path(CACHE_DIR, "timeseries", "forcing_analysis_assim", "*.parquet"),
    secondary_filepath=Path(CACHE_DIR, "timeseries", "forcing_medium_range", "*.parquet"),
    crosswalk_filepath=Path(CACHE_DIR, "geo", "huc10_huc10_crosswalk.parquet"),
    geometry_filepath=Path(CACHE_DIR, "geo", "huc10_geometry.parquet"),
)
# short range streamflow forecast evaluation files 
SRF_streamflow = dict(
    scenario_name="short_range",
    variable="streamflow",
    primary_filepath=MRF_streamflow["primary_filepath"],
    secondary_filepath=Path(CACHE_DIR, "timeseries", "short_range", "*.parquet"),
    crosswalk_filepath=MRF_streamflow["crosswalk_filepath"],
    geometry_filepath=MRF_streamflow["geometry_filepath"],
)
# medium range precip forecast evaluation files
SRF_forcing = dict(
    scenario_name="short_range",
    variable="precipitation",    
    primary_filepath=MRF_forcing["primary_filepath"],
    secondary_filepath=Path(CACHE_DIR, "timeseries", "forcing_short_range", "*.parquet"),
    crosswalk_filepath=MRF_forcing["crosswalk_filepath"],
    geometry_filepath=MRF_forcing["geometry_filepath"],
)
attribute_paths = dict(
    usgs_upstream_area=Path(CACHE_DIR, "geo", "usgs_attr_upstream_area.parquet"),
    usgs_ecoregions=Path(CACHE_DIR, "geo", "usgs_attr_ecoregions.parquet"),
    usgs_stream_order=Path(CACHE_DIR, "geo", "usgs_attr_stream_order.parquet"),
    usgs_huc_crosswalk=Path(CACHE_DIR, "geo", "usgs_huc12_crosswalk.parquet"),
)

# put the scenarios in a list for widget purposes
scenario_definitions = [MRF_streamflow, MRF_forcing, SRF_streamflow, SRF_forcing]

## general units ('english' or 'metric') to show in visualization
viz_units = "metric"

### First select the scenario and variable
Set up dropdown menus to select the specific scenario to evaluate based on the scenario name and variable

In [3]:
scenario_selector = du.get_scenario_selector(scenario_name_list=sorted(du.get_scenario_names(scenario_definitions)))  
variable_selector = du.get_variable_selector(variable_list=du.get_scenario_variables(scenario_definitions))   
pn.Row(scenario_selector, variable_selector)

Row
    [0] Select(name='Evaluation scenario', options=['medium_range', ...], value='medium_range', width_policy='fit')
    [1] Select(name='Evaluation variable', options=['precipitation', ...], value='streamflow', width_policy='fit')

### Then select the region, time period, characteristics and metrics of interest

Set up dropdowns and sliders to get the list or range of options and enable you to easily select a particular subset of data for metric calculations.

In [4]:
scenario = du.get_scenario(scenario_definitions, scenario_selector.value, variable_selector.value)
widgets = du.get_filter_widgets(scenario=scenario, include_widgets=[
    'value_time','reference_time','lead_time','huc2','threshold','stream_order','metrics'])

pn.Row(
    pn.Column(widgets['huc2'], widgets['stream_order'], widgets['threshold'], widgets['metrics']),
    pn.Spacer(width=50),    
    pn.Column(     
        pn.Spacer(height=10), widgets['value_time'],
        pn.Spacer(height=10), widgets['reference_time'],
        pn.Spacer(height=5), widgets['lead_time'],
    )
)

Checking value_time range in the parquet files


type: IO Error: No files found that match the pattern "C:\Users\kvanwerkhoven\shared\teehr-workshop\post-event-example\timeseries\usgs\*.parquet"

### Make selections

Using the menus and sliders above, make selections to evaluate a particular subset of the data.  Many more filters are possible here, these are just a few examples we anticipate may be commonly needed.

### Run the query
Run the cell below to execute a wrapper which builds the filter portion of the TEEHR query and then executes the query.  Experiment with the filter widget selections to see how the run time and query output change.  

In [5]:
%%time
gdf = du.run_teehr_query(
    query_type="metrics",
    scenario=scenario,
    huc_id=widgets['huc2'].value,
    order_limit=widgets['stream_order'].value,
    value_time_start=widgets['value_time'][1].value_start,    
    value_time_end=widgets['value_time'][1].value_end,    
    reference_time_start=widgets['reference_time'][1].value_start,    
    reference_time_end=widgets['reference_time'][1].value_end,
    group_by=['primary_location_id'],
    order_by=['primary_location_id'],
    value_min=widgets['threshold'].value,    
    include_metrics=widgets['metrics'].value,
    attribute_paths=attribute_paths,
    return_query=False,
)
display(gdf.head())

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,measurement_unit,primary_location_id,primary_maximum,secondary_maximum,max_value_delta,primary_max_value_time,secondary_max_value_time,max_value_timedelta,geometry
0,m3/s,usgs-10251290,0.005663,0.22,0.214337,2023-01-19 22:00:00,2023-01-15 18:00:00,-5 days +20:00:00,POINT (-116.23425 35.88569)
1,m3/s,usgs-10251300,13.931889,1.97,-11.961889,2023-01-15 05:00:00,2023-01-15 21:00:00,0 days 16:00:00,POINT (-116.23083 35.84944)
2,m3/s,usgs-10251330,1.449823,1.91,0.460177,2023-01-15 13:00:00,2023-01-15 22:00:00,0 days 09:00:00,POINT (-116.20764 35.79053)
3,m3/s,usgs-10251335,0.013875,4.57,4.556125,2023-01-15 07:00:00,2023-01-15 08:00:00,0 days 01:00:00,POINT (-116.19442 35.80094)
4,m3/s,usgs-10254050,0.097127,0.10,0.002873,2023-01-03 00:00:00,2023-01-03 01:00:00,0 days 01:00:00,POINT (-115.84333 33.44697)


CPU times: user 54.3 s, sys: 3.8 s, total: 58.1 s
Wall time: 15.4 s


### Create a simple map of results

In [6]:
gdf = gdf.to_crs("EPSG:3857")
gdf['relative_peak_error'] = gdf['max_value_delta']/gdf['primary_maximum']
gdf.hvplot.points(c='relative_peak_error', cmap=cc.CET_D1A[::-1], clim=(-1,1), width=600, height=400,
                             title="Relative Peak Error", size=10, xaxis = None, yaxis = None, tiles='CartoLight', 
                             hover_cols=['primary_location_id', 'primary_maximum'], cnorm='linear')


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (relative_peak_error,primary_location_id,primary_maximum)

### Choose a gage to explore more closely
Using the zoom and hover features on the map, select a gage and manually enter the ID in the cell below. (Gage selection by clicking on points in the map will be demonstrated in notebook 09).

In [7]:
usgs_id = 'usgs-11451715'

Using the wrapper again, build and run a TEEHR timeseries query to extract the streamflow timeseries.

In [8]:
flow_df = du.run_teehr_query(
    query_type="timeseries",
    scenario=scenario,
    location_id=usgs_id,
    order_limit=widgets['stream_order'].value,
    value_time_start=widgets['value_time'][1].value_start,    
    value_time_end=widgets['value_time'][1].value_end,    
    reference_time_start=widgets['reference_time'][1].value_start,    
    reference_time_end=widgets['reference_time'][1].value_end,
    value_min=widgets['threshold'].value,    
    attribute_paths=attribute_paths,
    return_query=False,
    include_geometry=False,
)
display(flow_df.head())

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,reference_time,value_time,secondary_location_id,secondary_value,configuration,measurement_unit,variable_name,primary_value,primary_location_id,lead_time
0,2023-01-01,2023-01-01 01:00:00,nwm22-8009213,13.480000,medium_range_mem1,m3/s,streamflow,7.164162,usgs-11451715,0 days 01:00:00
1,2023-01-01,2023-01-01 02:00:00,nwm22-8009213,18.359999,medium_range_mem1,m3/s,streamflow,7.164162,usgs-11451715,0 days 02:00:00
2,2023-01-01,2023-01-01 03:00:00,nwm22-8009213,21.449999,medium_range_mem1,m3/s,streamflow,7.022578,usgs-11451715,0 days 03:00:00
3,2023-01-01,2023-01-01 04:00:00,nwm22-8009213,21.650000,medium_range_mem1,m3/s,streamflow,7.022578,usgs-11451715,0 days 04:00:00
4,2023-01-01,2023-01-01 05:00:00,nwm22-8009213,20.740000,medium_range_mem1,m3/s,streamflow,7.305746,usgs-11451715,0 days 05:00:00


### Plot the timeseries by reference time
Loop through the time series, sequentially adding an additional curve to the holoviews overlay `hydrographs` and further overlay with the observed data

In [9]:
flow_df = flow_df.rename(columns={'primary_value':'obs_flow','secondary_value':'fcst_flow'})
ref_times = sorted(flow_df['reference_time'].unique())
cmap = cc.rainbow[::-1]
cstep = int(np.floor(len(cmap) / len(ref_times)))

flow_obs = flow_df.hvplot(x='value_time', y='obs_flow', label = 'observed', ylabel='Flow (cms)', color = 'black', line_width=4)   
hydrographs = flow_obs
for t, time in enumerate(ref_times):
    df_t = flow_df[flow_df['reference_time'] == time]
    ci = cmap[cstep * t]
    fcst = df_t.hvplot(x='value_time', y='fcst_flow', color = [ci])
    hydrographs = hydrographs * fcst

hydrographs * flow_obs

:Overlay
   .Curve.Observed.I  :Curve   [value_time]   (obs_flow)
   .Curve.I           :Curve   [value_time]   (fcst_flow)
   .Curve.II          :Curve   [value_time]   (fcst_flow)
   .Curve.III         :Curve   [value_time]   (fcst_flow)
   .Curve.IV          :Curve   [value_time]   (fcst_flow)
   .Curve.V           :Curve   [value_time]   (fcst_flow)
   .Curve.VI          :Curve   [value_time]   (fcst_flow)
   .Curve.VII         :Curve   [value_time]   (fcst_flow)
   .Curve.VIII        :Curve   [value_time]   (fcst_flow)
   .Curve.IX          :Curve   [value_time]   (fcst_flow)
   .Curve.X           :Curve   [value_time]   (fcst_flow)
   .Curve.XI          :Curve   [value_time]   (fcst_flow)
   .Curve.XII         :Curve   [value_time]   (fcst_flow)
   .Curve.XIII        :Curve   [value_time]   (fcst_flow)
   .Curve.XIV         :Curve   [value_time]   (fcst_flow)
   .Curve.XV          :Curve   [value_time]   (fcst_flow)
   .Curve.XVI         :Curve   [value_time]   (fcst_flow)
   .Curve.XVII        :Curve   [value_time]   (fcst_flow)
   .Curve.XVIII       :Curve   [value_time]   (fcst_flow)
   .Curve.XIX         :Curve   [value_time]   (fcst_flow)
   .Curve.XX          :Curve   [value_time]   (fcst_flow)
   .Curve.XXI         :Curve   [value_time]   (fcst_flow)
   .Curve.XXII        :Curve   [value_time]   (fcst_flow)
   .Curve.XXIII       :Curve   [value_time]   (fcst_flow)
   .Curve.XXIV        :Curve   [value_time]   (fcst_flow)
   .Curve.XXV         :Curve   [value_time]   (fcst_flow)
   .Curve.XXVI        :Curve   [value_time]   (fcst_flow)
   .Curve.XXVII       :Curve   [value_time]   (fcst_flow)
   .Curve.XXVIII      :Curve   [value_time]   (fcst_flow)
   .Curve.XXIX        :Curve   [value_time]   (fcst_flow)
   .Curve.XXX         :Curve   [value_time]   (fcst_flow)
   .Curve.XXXI        :Curve   [value_time]   (fcst_flow)
   .Curve.XXXII       :Curve   [value_time]   (fcst_flow)
   .Curve.XXXIII      :Curve   [value_time]   (fcst_flow)
   .Curve.XXXIV       :Curve   [value_time]   (fcst_flow)
   .Curve.XXXV        :Curve   [value_time]   (fcst_flow)
   .Curve.XXXVI       :Curve   [value_time]   (fcst_flow)
   .Curve.XXXVII      :Curve   [value_time]   (fcst_flow)
   .Curve.XXXVIII     :Curve   [value_time]   (fcst_flow)
   .Curve.XXXIX       :Curve   [value_time]   (fcst_flow)
   .Curve.XL          :Curve   [value_time]   (fcst_flow)
   .Curve.XLI         :Curve   [value_time]   (fcst_flow)
   .Curve.Observed.II :Curve   [value_time]   (obs_flow)

### Now let's get the precipitation analysis and forecasts as well

For basic comparison purposes, find and extract the mean areal precipitation time series for the HUC10 containing the above gage.  Eventually we will want to calculate the effective mean areal timeseries for the entire upstream drainage basin associated with a gage or other point.

In [10]:
# read the usgs-huc crosswalk, get the HUC10 containing the above gage
cross = pd.read_parquet(attribute_paths['usgs_huc_crosswalk'])
huc12_id = cross.loc[cross['primary_location_id']==usgs_id, 'secondary_location_id'].iloc[0]
huc10_id = "-".join(['huc10', huc12_id.split("-")[1][:10]])

pcp_df = du.run_teehr_query(
    query_type="timeseries",
    scenario=MRF_forcing,
    location_id=huc10_id,
    value_time_start=widgets['value_time'][1].value_start,    
    value_time_end=widgets['value_time'][1].value_end,    
    reference_time_start=widgets['reference_time'][1].value_start,    
    reference_time_end=widgets['reference_time'][1].value_end,
    attribute_paths=attribute_paths,
    return_query=False,
    include_geometry=False,
)
display(pcp_df.head())

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,reference_time,value_time,secondary_location_id,secondary_value,configuration,measurement_unit,variable_name,primary_value,primary_location_id,lead_time
0,2023-01-01,2023-01-01 01:00:00,huc10-1802011605,1.999930,forcing_medium_range,mm/hr,precipitation_rate,0.972265,huc10-1802011605,0 days 01:00:00
1,2023-01-01,2023-01-01 02:00:00,huc10-1802011605,0.983824,forcing_medium_range,mm/hr,precipitation_rate,1.308413,huc10-1802011605,0 days 02:00:00
2,2023-01-01,2023-01-01 03:00:00,huc10-1802011605,0.086153,forcing_medium_range,mm/hr,precipitation_rate,0.000000,huc10-1802011605,0 days 03:00:00
3,2023-01-01,2023-01-01 04:00:00,huc10-1802011605,0.091350,forcing_medium_range,mm/hr,precipitation_rate,0.000000,huc10-1802011605,0 days 04:00:00
4,2023-01-01,2023-01-01 05:00:00,huc10-1802011605,0.015139,forcing_medium_range,mm/hr,precipitation_rate,0.000000,huc10-1802011605,0 days 05:00:00


Now let's create linked precipitation and streamflow time series overlay plots

In [11]:
pcp_df = pcp_df.rename(columns={'primary_value':'obs_pcp','secondary_value':'fcst_pcp'})
pcp_obs = pcp_df.hvplot(x='value_time', y='obs_pcp', label='observed', ylabel='Precip (mm)', color = 'black', line_width=3)   
hyetograph = pcp_obs
for t, time in enumerate(ref_times):
    df_t = pcp_df[pcp_df['reference_time'] == time]
    ci = cmap[cstep * t]
    fcst = df_t.hvplot(x='value_time', y='fcst_pcp', color = [ci])
    hyetograph = hyetograph * fcst

((hyetograph * pcp_obs) + (hydrographs * flow_obs)).cols(1)

:Layout
   .Overlay.I  :Overlay
      .Curve.Observed.I  :Curve   [value_time]   (obs_pcp)
      .Curve.I           :Curve   [value_time]   (fcst_pcp)
      .Curve.II          :Curve   [value_time]   (fcst_pcp)
      .Curve.III         :Curve   [value_time]   (fcst_pcp)
      .Curve.IV          :Curve   [value_time]   (fcst_pcp)
      .Curve.V           :Curve   [value_time]   (fcst_pcp)
      .Curve.VI          :Curve   [value_time]   (fcst_pcp)
      .Curve.VII         :Curve   [value_time]   (fcst_pcp)
      .Curve.VIII        :Curve   [value_time]   (fcst_pcp)
      .Curve.IX          :Curve   [value_time]   (fcst_pcp)
      .Curve.X           :Curve   [value_time]   (fcst_pcp)
      .Curve.XI          :Curve   [value_time]   (fcst_pcp)
      .Curve.XII         :Curve   [value_time]   (fcst_pcp)
      .Curve.XIII        :Curve   [value_time]   (fcst_pcp)
      .Curve.XIV         :Curve   [value_time]   (fcst_pcp)
      .Curve.XV          :Curve   [value_time]   (fcst_pcp)
      .Curve.XVI         :Curve   [value_time]   (fcst_pcp)
      .Curve.XVII        :Curve   [value_time]   (fcst_pcp)
      .Curve.XVIII       :Curve   [value_time]   (fcst_pcp)
      .Curve.XIX         :Curve   [value_time]   (fcst_pcp)
      .Curve.XX          :Curve   [value_time]   (fcst_pcp)
      .Curve.XXI         :Curve   [value_time]   (fcst_pcp)
      .Curve.XXII        :Curve   [value_time]   (fcst_pcp)
      .Curve.XXIII       :Curve   [value_time]   (fcst_pcp)
      .Curve.XXIV        :Curve   [value_time]   (fcst_pcp)
      .Curve.XXV         :Curve   [value_time]   (fcst_pcp)
      .Curve.XXVI        :Curve   [value_time]   (fcst_pcp)
      .Curve.XXVII       :Curve   [value_time]   (fcst_pcp)
      .Curve.XXVIII      :Curve   [value_time]   (fcst_pcp)
      .Curve.XXIX        :Curve   [value_time]   (fcst_pcp)
      .Curve.XXX         :Curve   [value_time]   (fcst_pcp)
      .Curve.XXXI        :Curve   [value_time]   (fcst_pcp)
      .Curve.XXXII       :Curve   [value_time]   (fcst_pcp)
      .Curve.XXXIII      :Curve   [value_time]   (fcst_pcp)
      .Curve.XXXIV       :Curve   [value_time]   (fcst_pcp)
      .Curve.XXXV        :Curve   [value_time]   (fcst_pcp)
      .Curve.XXXVI       :Curve   [value_time]   (fcst_pcp)
      .Curve.XXXVII      :Curve   [value_time]   (fcst_pcp)
      .Curve.XXXVIII     :Curve   [value_time]   (fcst_pcp)
      .Curve.XXXIX       :Curve   [value_time]   (fcst_pcp)
      .Curve.XL          :Curve   [value_time]   (fcst_pcp)
      .Curve.XLI         :Curve   [value_time]   (fcst_pcp)
      .Curve.Observed.II :Curve   [value_time]   (obs_pcp)
   .Overlay.II :Overlay
      .Curve.Observed.I  :Curve   [value_time]   (obs_flow)
      .Curve.I           :Curve   [value_time]   (fcst_flow)
      .Curve.II          :Curve   [value_time]   (fcst_flow)
      .Curve.III         :Curve   [value_time]   (fcst_flow)
      .Curve.IV          :Curve   [value_time]   (fcst_flow)
      .Curve.V           :Curve   [value_time]   (fcst_flow)
      .Curve.VI          :Curve   [value_time]   (fcst_flow)
      .Curve.VII         :Curve   [value_time]   (fcst_flow)
      .Curve.VIII        :Curve   [value_time]   (fcst_flow)
      .Curve.IX          :Curve   [value_time]   (fcst_flow)
      .Curve.X           :Curve   [value_time]   (fcst_flow)
      .Curve.XI          :Curve   [value_time]   (fcst_flow)
      .Curve.XII         :Curve   [value_time]   (fcst_flow)
      .Curve.XIII        :Curve   [value_time]   (fcst_flow)
      .Curve.XIV         :Curve   [value_time]   (fcst_flow)
      .Curve.XV          :Curve   [value_time]   (fcst_flow)
      .Curve.XVI         :Curve   [value_time]   (fcst_flow)
      .Curve.XVII        :Curve   [value_time]   (fcst_flow)
      .Curve.XVIII       :Curve   [value_time]   (fcst_flow)
      .Curve.XIX         :Curve   [value_time]   (fcst_flow)
      .Curve.XX          :Curve   [value_time]   (fcst_flow)
      .Curve.XXI         :Curve   [value_time]   (fcst_flow)
      .Curve.XXII        